In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 

In [2]:
target_labels = ['Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [5]:
# Load the images and labels
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(target_labels.index(label))

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [8]:
X[0].shape

(128, 128, 3)

In [9]:
X[0].shape

(128, 128, 3)

In [10]:
X.shape

(7222, 128, 128, 3)

In [11]:
y.shape

(7222, 4)

In [12]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_train.shape

(5777, 128, 128, 3)

In [14]:
X_test.shape

(1445, 128, 128, 3)

In [15]:
y_train.shape

(5777, 4)

In [16]:
y_test.shape

(1445, 4)

In [17]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [18]:
len(target_labels)

4

In [19]:
cnn_model = Sequential()
cnn_model.add(Conv2D(128, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(16,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [20]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 128)      0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 63, 63, 128)      512       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                        

In [21]:
# Train the CNN model
cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-30 21:50:31.048851: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


181/181 [==============================] - 95s 523ms/step - loss: 0.5128 - accuracy: 0.8025 - val_loss: 2.9867 - val_accuracy: 0.4235
Epoch 2/10
181/181 [==============================] - 105s 579ms/step - loss: 0.2245 - accuracy: 0.9148 - val_loss: 1.4368 - val_accuracy: 0.5744
Epoch 3/10
181/181 [==============================] - 124s 684ms/step - loss: 0.1609 - accuracy: 0.9411 - val_loss: 0.2248 - val_accuracy: 0.9170
Epoch 4/10
181/181 [==============================] - 117s 645ms/step - loss: 0.1216 - accuracy: 0.9571 - val_loss: 0.3432 - val_accuracy: 0.8754
Epoch 5/10
181/181 [==============================] - 110s 610ms/step - loss: 0.0939 - accuracy: 0.9678 - val_loss: 0.4662 - val_accuracy: 0.8401
Epoch 6/10
181/181 [==============================] - 107s 592ms/step - loss: 0.0819 - accuracy: 0.9714 - val_loss: 0.0897 - val_accuracy: 0.9675
Epoch 7/10
181/181 [==============================] - 147s 813ms/step - loss: 0.0711 - accuracy: 0.9742 - val_loss: 0.6414 - val_accurac

In [22]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 98.75%


In [23]:
cnn_scores

[0.03694421797990799, 0.9875432252883911]

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

46/46 [==============================] - 9s 198ms/step
Accuracy: 98.75%
Precision: 98.76%
Recall: 98.75%
F1 score: 98.76%


In [25]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

181/181 [==============================] - 35s 194ms/step


# tes rf non lib

In [26]:
import time

In [27]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [28]:
RandomForest

RF_Manual.RandomForest

In [29]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 29.894893884658813 seconds


In [30]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

46/46 [==============================] - 9s 198ms/step
Random Forest Classifier Accuracy: 98.89%
Random Forest Classifier Precision: 0.99
Random Forest Classifier Recall: 0.99
Random Forest Classifier F1 Score: 0.99


# tes

In [31]:
#Label assignment
label=['Grape___Black_rot', 
       'Grape___Esca_(Black_Measles)', 
       'Grape___healthy', 
       'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)']

In [32]:
from tensorflow.keras.preprocessing import image

In [33]:
def testing_rf(path, rf_classifier):
    test_image = image.load_img(path, target_size=(128, 128))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    rf_features = cnn_model.predict(test_image)
    rf_features = rf_features.reshape(rf_features.shape[0], -1)
    result = rf_classifier.predict(rf_features)

    label2 = label[result[0]]
    print(f"Result is --> {result}")
    print(f"Your leaf disease is --> {label2}")

# testing(input("Enter your image path-: "))
path = 'data_penyakit/anggur/grape-black-rot-2.JPG'
testing_rf(path, rf_classifier)

1/1 [==============================] - 0s 39ms/step
Result is --> [0]
Your leaf disease is --> Grape___Black_rot


In [34]:
import os

def testing_folder_rf(path, rf_classifier):
    test_image = image.load_img(path, target_size=(128, 128))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    rf_features = cnn_model.predict(test_image)
    rf_features = rf_features.reshape(rf_features.shape[0], -1)
    result = rf_classifier.predict(rf_features)

    label2 = label[result[0]]
    return label2

folder_path = 'data_penyakit/anggur'
results = []

for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    print(f"Testing image: {filename}")
    try:
        label2 = testing_folder_rf(image_path, rf_classifier)
        print(f"Your leaf disease is --> {label2}")
        results.append({"filename": filename, "disease": label2})  # append result to list
    except IOError:
        print(f"Cannot open {filename}")
    print("\n")

# df_rf = pd.DataFrame(results)

# Save the dataframe into a CSV file
# df_rf.to_csv('results-rf-anggur.csv', index=False)


Testing image: .DS_Store
Cannot open .DS_Store


Testing image: grape-healthy-3.JPG
1/1 [==============================] - 0s 22ms/step
Your leaf disease is --> Grape___healthy


Testing image: grape-black-rot-3.JPG
1/1 [==============================] - 0s 23ms/step
Your leaf disease is --> Grape___Black_rot


Testing image: grape-black-rot-2.JPG
1/1 [==============================] - 0s 32ms/step
Your leaf disease is --> Grape___Black_rot


Testing image: grape-healthy-2.JPG
1/1 [==============================] - 0s 28ms/step
Your leaf disease is --> Grape___healthy


Testing image: grape-black-rot-1.JPG
1/1 [==============================] - 0s 21ms/step
Your leaf disease is --> Grape___Black_rot


Testing image: grape-healthy-1.JPG
1/1 [==============================] - 0s 25ms/step
Your leaf disease is --> Grape___healthy


Testing image: grape-healthy-5.JPG
1/1 [==============================] - 0s 20ms/step
Your leaf disease is --> Grape___healthy


Testing image: grape-black-ro